In [12]:
import pandas as pd
import json
import numpy as np
import time
import nltk;
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
!pip install en
from collections import defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shwethailango/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### configuration figures

In [13]:
#fileName
file_name = "Response.json"
#targetColumn
feature_name= 'text'

#hyperparameters & k-number of topics
k = 20
init = 'k-means++'
max_iter = 700
n_init = 10

##### pre-processing: depends on input file

In [14]:
f = open(file_name) 
df1=json.load(f)
keys_to_extract = [feature_name]
new=[]

for i in range (0,len(df1['responseList'])):
    d=[{key: df1['responseList'][i][key] for key in keys_to_extract}]
    a=pd.DataFrame.from_dict(d) 
    new.append(a)
train_df=pd.concat(new, ignore_index=True)   


###### Complete function to generate topics with dataframe as the input

In [15]:
def generateTopics(train_df):
    
    def textCleaning(train_df):
        clean_df = train_df.dropna()
        temp = clean_df[feature_name].str.split('.')
        clean_df = clean_df.reindex(clean_df.index.repeat(temp.apply(len)))
        clean_df['feature_clean'] = np.hstack(temp)
        new_df = clean_df.copy().reset_index()
        temp = new_df['feature_clean'].str.split(',')
        new_df = new_df.reindex(new_df.index.repeat(temp.apply(len)))
        new_df['feature_clean_all'] = np.hstack(temp)
        my_df = new_df['feature_clean_all'].reset_index()
        your_df = pd.DataFrame(my_df)
        your_df['feature_clean_all'].replace('', np.nan, inplace=True)
        your_df.dropna(subset=['feature_clean_all'], inplace=True)
        your_df['feature_clean_all'].replace(' ', np.nan, inplace=True)
        your_df.dropna(subset=['feature_clean_all'], inplace=True)
        return clean_df,your_df

    class TextPreProcessor():

        NLP = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
        parser = English()
        noisy_pos_tags = ['PROP']
        min_token_length = 2


        @staticmethod
        def stopWords():
            spacy_stopwords = TextPreProcessor.NLP.Defaults.stop_words

            stop_words = stopwords.words('english')
            stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'need', 'able', 'bmc', 'com'])

            stop_words.extend(spacy_stopwords)
            return stop_words

        @staticmethod
        def removeStopWords(data):
            STOP_WORDS = TextPreProcessor.stopWords()
            cleanedData = []
            for sent in data:
                sent_cleaned = ""
                for word in sent.split():
                    if word not in STOP_WORDS and word not in string.punctuation and len(word) > TextPreProcessor.min_token_length:
                        sent_cleaned += word
                        sent_cleaned += " "
                cleanedData.append(sent_cleaned)
            return cleanedData

        @staticmethod
        def removePunctuation(data):
            return [sent.translate(sent.maketrans(string.punctuation, ' '*len(string.punctuation))) for sent in data]

        @staticmethod
        def removeHostName(data):
            cleanedData = []
            for sent in data:
                sent_cleaned = ""
                for word in sent.split():
                    sent_cleaned += re.sub(r'^([A-Za-z0-9]\.|[A-Za-z0-9][A-Za-z0-9-]{0,61}[A-Za-z0-9]\.){1,3}[A-Za-z]{2,6}$', '', word)
                    sent_cleaned += " "
                cleanedData.append(sent_cleaned)
            return cleanedData

        @staticmethod
        def removeIPAddress(data):
            cleanedData = []
            for sent in data:
                sent_cleaned = ""
                for word in sent.split():
                    sent_cleaned += re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$', '', word)
                    sent_cleaned += " "
                cleanedData.append(sent_cleaned)
            return cleanedData

        @staticmethod
        def removeEmails(data):
            return [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

        @staticmethod
        def removeNewLineCharacters(data):
            return [re.sub(r'\s+', ' ', sent) for sent in data]

        @staticmethod
        def removeDigits(data):
            return [re.sub(r'\d', ' ', sent) for sent in data]

        @staticmethod
        def toLower(data):
            return [sent.lower() for sent in data]

        @staticmethod
        def spacy_tokenizer(sent):
            tokens = TextPreProcessor.parser(sent)
            tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
            tokens = [tok for tok in tokens if (tok not in STOP_WORDS and tok not in string.punctuation)]     
            return tokens

        def lemmatization(data, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
            stopWords = TextPreProcessor.stopWords()
            texts_out = []
            tokens_out = []
            for sent in data:
                #print(sent)
                doc = TextPreProcessor.NLP(sent) 
                lemmed_string =''
                tokens = []
                for word in doc:
                    if word.pos_ in allowed_postags:
                        if word.lemma_ == '-PRON-' or word.lemma_ in stopWords: 
                            continue 
                        else: 
                            lemmed_string = lemmed_string + ' ' + word.lemma_
                            tokens.append(word.lemma_)
                #print(lemmed_string)
                texts_out.append(lemmed_string.strip())
                tokens_out.append(tokens)
            return tokens_out

        def cleanup(data):
            data = TextPreProcessor.toLower(data)
            data = TextPreProcessor.removeHostName(data)
            data = TextPreProcessor.removeIPAddress(data)
            data = TextPreProcessor.removeNewLineCharacters(data)
            data = TextPreProcessor.removePunctuation(data)
            data = TextPreProcessor.removeDigits(data)
            data = TextPreProcessor.removeEmails(data)
            data = TextPreProcessor.removeStopWords(data)
            data = TextPreProcessor.lemmatization(data)
            return data

        def __init__(self, df, columns, combine = False, dropCombinedColumns = False):
            self.df = df
            self.columns = columns
            self.combine = combine
            self.dropCombinedColumns = dropCombinedColumns
            self.process()

        def process(self):  
            columnToProcess = self.columns                   
            if self.combine:
                self.df['combined'] = self.df[self.columns].apply(
                                            lambda x: ','.join(x.astype(str)), axis=1)
                if dropCombinedColumns:
                    self.df = self.df.drop(self.columns, axis=1)
                    columnToProcess = ["combined"]
                else:
                    columnsToProcess.append("combined")
            for col in columnToProcess:

                colName = col + "_clean"
                self.df[colName] = TextPreProcessor.cleanup(self.df[col])
            return self.df

    def process(data_df):
        preProcessor = TextPreProcessor(your_df, ['feature_clean_all'])
        clean_df = preProcessor.df
        return clean_df     


    def k_mean_distance(data, centroid_matrix, i_centroid, cluster_labels):
        no_of_rec = data[cluster_labels == i_centroid].shape[0]
        distances = [np.linalg.norm(x - centroid_matrix) for x in data[cluster_labels == i_centroid]]
        wcss = sum(i * i for i in distances)
        if no_of_rec != 0:
            mean = wcss/no_of_rec
        else:
            mean = 0
     
        return round(wcss, 2), no_of_rec, mean

    def get_goodness_score(clusters, centroids, emb_matrix):
        score = []
        record_list = []
        mean_list = []
        for i, centroid_matrix in enumerate(centroids):
            cs, no_of_rec, mean = k_mean_distance(emb_matrix, centroid_matrix, i, clusters)
            score.append(cs)
            record_list.append(no_of_rec)
            mean_list.append(round(mean,2))
        return score, record_list, mean_list


    #text-cleaning    
    clean_df,your_df=textCleaning(train_df) 

    #text-preprocessing
    clean_df = process(clean_df)
    msg_vec = clean_df['feature_clean_all_clean']
    msg_vec = msg_vec.apply(lambda x: str(x).replace('[','').replace(']','').replace(',','')) 
    msg_vec = msg_vec.apply(lambda x: x.replace('\'',''))
    clean_df['Description_clean'] = msg_vec

    #text-vectorising
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(msg_vec)

    #clustering
    #start_time = time.time()
    true_k = k
    k_means = KMeans(n_clusters=true_k, init=init, max_iter=max_iter, n_init=n_init)
    kmeans_fit = k_means.fit(X)
    clusters = k_means.fit_predict(X)
    centroids = k_means.cluster_centers_

    #scores for reference
    score, record_list, mean_list = get_goodness_score(clusters, centroids, X)
    label = k_means.labels_
    overall_score = silhouette_score(X, label, metric='euclidean')

    # topic modeling
    val = []
    order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(k):
        topic = ''
        for ind in order_centroids[i, :3]:
            topic = str(topic + ' ' + terms[ind]).strip()
        val.append(topic.replace(' ', '-'))

    #loading topics into a dataframe
    cluster_results = pd.DataFrame()
    for slot in clean_df.columns:
        cluster_results[slot] = clean_df[slot]
    kmeans_fit.labels_.shape
    cluster_results['cluster_id'] = kmeans_fit.labels_
    cluster_results['cluster_labels'] = cluster_results['cluster_id'].apply(lambda x: val[x])
    cluster_results['goodness_score'] = cluster_results['cluster_id'].apply(lambda x: mean_list[x])
    cluster_results['cluster_id'] = cluster_results['cluster_id'].apply(lambda x: str(x))   
    cluster_results.sort_values(by='cluster_id', inplace=True)

    #output response with labelName and review-text
    finalDict=defaultdict(list)
    j=1
    for i in cluster_results.cluster_labels.unique():
        d={}
        d['LabelName']=i
        reviewList=cluster_results['feature_clean_all'][cluster_results.cluster_labels==i]
        d['Reviews']=list(set(reviewList))
        finalDict[j]=d
        j=j+1


    return dict(finalDict)

###### output is stored in responseDict

In [16]:
responseDict=generateTopics(train_df)

In [17]:
display(responseDict)

{1: {'LabelName': 'decor-later-hotel',
  'Reviews': [' i was there to celebrate my fiance birthday which i wanted to be a surprise',
   ' not praise worthy',
   'help with smile!',
   ' The entry to the Hotel oddly starts with a descent from the Main entrance gate but into the Paradise walking by the side of small temple',
   ' but there was plenty to choose from',
   'Located near Mall road',
   ' fantastic spread for breakfast ',
   ' rupees after reaching the hotel I found they have provided me a dormitory ',
   'one is not informed about the limited availability of hot water at check-in and it is even worse to find that out in the midst of a bath',
   ' It is way too over priced',
   'inside n outside decor is mesmerising',
   ' MAIN BAHUT PARESHAN HUA',
   ' Decor of the hotel with its grandeur has won my heart',
   'Bed very uncomfortable\n',
   ' Two things which did not go well for me: 1) Mattress was soft / more like had lived its life and 2) the treadmill and cycle in gym wer